In [1]:
import torch, torchvision
import torch.optim as optim
import torch.nn as nn
import os
#import oil.augLayers as augLayers
from oil.model_trainers.cGan import cGan
from oil.model_trainers.classifier import Classifier
from oil.datasetup.datasets import CIFAR10, C10augLayers
from oil.datasetup.dataloaders import getLabLoader
#from oil.architectures.networkparts import layer13,ConvSmallNWN
from oil.architectures import conditionalgan
from oil.utils.utils import cosLr, loader_to,imap,islice


train_epochs = 200
loader_config =     {'amnt_dev':0,'lab_BS':64,'dataseed':0,'num_workers':1}
trainer_config = {'n_disc':5}
trainer_config['log_dir'] = os.path.expanduser('~/tb-experiments/cgan_k128_2/')
sched_config =      {'cycle_length':train_epochs,'cycle_mult':1}

device = torch.device('cuda')
G = conditionalgan.Generator(num_classes=10).to(device)
D = conditionalgan.Discriminator(num_classes=10).to(device)
trainset, testset = CIFAR10(False, '~/datasets/cifar10/',gan_normalize=True)

dataloaders = {}
dataloaders['train'], _ = getLabLoader(trainset,**loader_config)
imgs_only  = imap(lambda z: z[0], dataloaders['train'])
dataloaders['dev'] = islice(imgs_only,5000//loader_config['lab_BS'])
dataloaders = {k: loader_to(device)(v) for k,v in dataloaders.items()}

lr_sched = cosLr(**sched_config)
trainer = cGan(G,dataloaders,D=D,**trainer_config)

Files already downloaded and verified
Files already downloaded and verified
Creating Train, Dev split         with 50000 Train and 0 Dev


In [2]:
trainer.load_checkpoint("/home/marc/tb-experiments/cgan_k128_2/checkpoints/c.0.ckpt")

=> loading checkpoint '/home/marc/tb-experiments/cgan_k128_2/checkpoints/c.0.ckpt'


In [3]:
L = trainer.as_dataloader(N=10000,bs=64)

In [7]:
from oil.utils.metrics import get_official_FID
from oil.model_trainers.gan import GanLoader
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
L = GanLoader(trainer.G,N=10000)

In [10]:
for img in L:
    break

In [11]:
img.shape

torch.Size([64, 3, 32, 32])

In [9]:
L.write_imgs(os.path.expanduser("~/pristine-ml/oil/utils/")+'temp/')

/home/marc/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


ValueError: Channel axis dimension is not valid.

In [ ]:
trainer.super().state_dict()

In [ ]:
d  = {'a':1}; b={'a':2}

In [ ]:
{**d,**b}

In [ ]:
q